In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 85.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio import features
import numpy as np

In [ ]:
!tree /content/drive/MyDrive


/bin/bash: line 1: tree: command not found


In [ ]:
from pathlib import Path
img_path = Path('/content/drive/MyDrive/mlp_rawdata/tif_img')

img_names = [f.name for f in img_path.iterdir() if f.is_file()]


In [ ]:
gpkg_path = '/content/drive/MyDrive/mlp_rawdata/TN_MAPINDX_5K.gpkg'

gpkg = gpd.read_file(gpkg_path)

gpkg = gpkg.to_crs(epsg=5186)


In [ ]:
gpkg[gpkg['MAPIDCD_NO']=='37608078'].geometry.total_bounds

array([193372.51997588, 550054.67002869, 195583.15009928, 552830.8499952 ])

In [ ]:
shp_path = '/content/drive/MyDrive/mlp_rawdata/AL_D010_11_20240404/AL_D010_11_20240404.shp'
shp = gpd.read_file(shp_path, encoding='cp949')
shp.head()

,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A20,A21,A22,A23,A24,A25,A26,A27,A28,geometry
0,32,1991201839054527769900000000,1111017500107040000,1111017500,서울특별시 종로구 숭인동,704,1,일반,01000,단독주택,...,N,B00100000000T30Z9,2024-04-01,11110,None,None,3.0,0.0,2017-05-30,"POLYGON ((201913.638 553085.323, 201912.228 55..."
1,34,1967201343844527773800000000,1111017500100560024,1111017500,서울특별시 종로구 숭인동,56-24,1,일반,01000,단독주택,...,N,B00100000000T311B,2024-04-01,11110,None,None,3.0,0.0,2018-06-19,"POLYGON ((201413.758 553078.463, 201410.198 55..."
2,36,1962201375194527742000000000,1111017500100560054,1111017500,서울특별시 종로구 숭인동,56-54,1,일반,01000,단독주택,...,N,B00100000000T313D,2024-04-01,11110,None,None,2.0,0.0,2017-02-24,"POLYGON ((201442.598 553075.703, 201442.298 55..."
3,37,1979201388484527710100000000,1111017500100560053,1111017500,서울특별시 종로구 숭인동,56-53,1,일반,01000,단독주택,...,N,B00100000000T314E,2024-04-01,11110,None,None,2.0,1.0,2018-11-06,"POLYGON ((201458.568 553069.403, 201453.548 55..."
4,38,0000200468294527764000000000,1111016500100280011,1111016500,서울특별시 종로구 이화동,28-11,1,일반,None,None,...,None,B00100000000T315F,2024-04-01,11110,None,None,0.0,0.0,None,"POLYGON ((200540.538 553080.543, 200536.438 55..."


In [14]:
for img in img_names:

  map_number = img.split("_")[2].split(".")[0]
  map = gpkg[gpkg['MAPIDCD_NO'] == map_number]

  bounds = map.total_bounds
  pixel = 0.25
  print(bounds)

  width = int((bounds[2] - bounds[0])/ pixel)
  height = int((bounds[3] - bounds[1]) / pixel)

  transform = rasterio.transform.from_origin(bounds[0], bounds[3], pixel, pixel)

  shapes = ((geom, i+1) for i, geom in enumerate(shp.geometry))
  raterized = features.rasterize(
      shapes = shapes,
      out_shape = (height, width),
      transform = transform
  )
  np_path = '/content/drive/MyDrive/mlp_rawdata/label_np/' + map_number + '.npy'

  np.save(np_path, raterized)
  print(np_path, ': 저장', width, height)





[191163.34997698 550056.14014061 193374.7301323  552832.91013482]
/content/drive/MyDrive/mlp_rawdata/label_np/37608077.npy : 저장 8845 11107
[188954.19000466 550058.1999555  191166.31012801 552835.54996823]
/content/drive/MyDrive/mlp_rawdata/label_np/37608076.npy : 저장 8848 11109
[186745.03000216 550060.83997073 188957.88013828 552838.77999988]
/content/drive/MyDrive/mlp_rawdata/label_np/37608075.npy : 저장 8851 11111
[188950.49999141 547283.49995049 191163.34997698 550060.83997073]
/content/drive/MyDrive/mlp_rawdata/label_np/37608086.npy : 저장 8851 11109
[195581.67994324 550053.79007986 197791.58003364 552829.38001192]
/content/drive/MyDrive/mlp_rawdata/label_np/37608079.npy : 저장 8839 11102
[191166.31012801 552830.8499952  193376.95006839 555607.61997626]
/content/drive/MyDrive/mlp_rawdata/label_np/37608067.npy : 저장 8842 11107
[188957.88013828 552832.91013482 191169.26010705 555610.27009141]
/content/drive/MyDrive/mlp_rawdata/label_np/37608066.npy : 저장 8845 11109
[186749.46011474 552835.549

In [ ]:
print(height, width)